In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sbpy.calib import Sun
from astropy.io import fits

from lvmdrp.core.constants import CON_LAMPS, SPEC_CHANNELS
from lvmdrp.utils import metadata as md
from lvmdrp import config, log, path, __version__ as drpver
from lvmdrp.functions import imageMethod, rssMethod
from lvmdrp.core import tracemask, rss

plt.style.use("seaborn-v0_8-talk")

In [2]:
flats = md.get_metadata(kind="master", imagetyp="flat")
flats = flats.loc[flats.name.str.contains("ldls") | flats.name.str.contains("quartz")]
flats.sort_values(by="camera", inplace=True)
flats

[INFO]: loading metadata store at /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/master_metadata.hdf5
[INFO]: found 63 frames in store '/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/master_metadata.hdf5'
[INFO]: filtering by imagetyp = 'flat'
[INFO]: number of frames after filtering 9
[INFO]: total number of frames found 9


,tileid,mjd,rmjd,imagetyp,spec,camera,exptime,neon,hgne,krypton,...,argon,ldls,quartz,quality,qual,stage,status,drpqual,nframes,name
3,1111,60147,60147,flat,sp1,b1,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,2,lvm-mflat_ldls-b1
5,1111,60147,60147,flat,sp2,b2,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,2,lvm-mflat_ldls-b2
1,1111,60147,60147,flat,sp3,b3,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,2,lvm-mflat_ldls-b3
7,1111,60147,60147,flat,sp1,r1,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,2,lvm-mflat_ldls-r1
2,1111,60147,60147,flat,sp2,r2,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,2,lvm-mflat_ldls-r2
6,1111,60147,60147,flat,sp3,r3,150.0,False,False,False,...,False,True,False,excellent,0,1,0,0,2,lvm-mflat_ldls-r3
4,1111,60147,60147,flat,sp1,z1,10.0,False,False,False,...,False,False,True,excellent,0,1,0,0,2,lvm-mflat_quartz-z1
0,1111,60147,60147,flat,sp2,z2,10.0,False,False,False,...,False,False,True,excellent,0,1,0,0,2,lvm-mflat_quartz-z2
8,1111,60147,60147,flat,sp3,z3,10.0,False,False,False,...,False,False,True,excellent,0,1,0,0,2,lvm-mflat_quartz-z3


In [3]:
# extract fiber flats using traces
lamps = [lamp.lower() for lamp in CON_LAMPS]
cameras = sorted(set(flats.camera))
mfiberflats = []
wflats = []
for _, flat in flats.iterrows():
    # get lamp used
    if flat.camera[0] != "b":
        continue
    lamp = flat[lamps].astype(int).idxmax()
    
    # define paths
    mflat = path.full("lvm_master", drpver=drpver, kind=f"mflat_{lamp}", **flat.to_dict())
    xflat = path.full("lvm_master", drpver=drpver, kind=f"xmflat_{lamp}", **flat.to_dict())
    wflat = path.full("lvm_master", drpver=drpver, kind=f"wmflat_{lamp}", **flat.to_dict())
    mfiberflat = path.full("lvm_master", drpver=drpver, kind=f"mfiberflat", **flat.to_dict())
    # define traces path
    mtrace = os.path.join("data_calib", f"lvm-mtrace-{flat.camera}.fits")
    mwave = os.path.join("data_calib", f"lvm-mwave_{'hgne' if flat.camera[0] == 'b' else 'neon'}-{flat.camera}.fits")
    mlsf = os.path.join("data_calib", f"lvm-mlsf_{'hgne' if flat.camera[0] == 'b' else 'neon'}-{flat.camera}.fits")
    iwave, fwave = SPEC_CHANNELS[flat.camera[0]]

    # extract fiberflat
    # imageMethod.extract_spectra(in_image=mflat, out_rss=xflat, in_trace=mtrace, method="aperture", aperture=3)
    # rssMethod.create_pixel_table(in_rss=xflat, out_rss=wflat, arc_wave=mwave, arc_fwhm=mlsf)
    # rssMethod.resample_wavelength(in_rss=wflat, out_rss=wflat, start_wave=iwave, end_wave=fwave, disp_pix=0.5, method="linear", err_sim=10, parallel=0, extrapolate=False)
    
    # compute fiberflats
    # rssMethod.create_fiberflat(in_rsss=wflat, out_rss=mfiberflat, median_box=0, gaussian_kernel=2.5, display_plots=True)
    # mfiberflats.append(mfiberflat)
    
    wflats.append(wflat)

In [4]:
for channel in "brz":
    wflats, mfiberflats = [], []
    for _, flat in flats.iterrows():
        if flat.camera[0] != channel: continue
        lamp = flat[lamps].astype(int).idxmax()
        wflats.append(path.full("lvm_master", drpver=drpver, kind=f"wmflat_{lamp}", **flat.to_dict()))
        mfiberflats.append(path.full("lvm_master", drpver=drpver, kind=f"mfiberflat", **flat.to_dict()))

    pars = flat.to_dict()
    pars["camera"] = channel
    mfiberflat = path.full("lvm_master", drpver=drpver, kind=f"mfiberflat", **pars)
    rssMethod.create_fiberflat(in_rsss=wflats, out_rsss=mfiberflats, median_box=0, gaussian_kernel=2.5, display_plots=True)

[INFO]: reading continuum exposure from lvm-wmflat_ldls-b1.fits
[INFO]: reading continuum exposure from lvm-wmflat_ldls-b2.fits
[INFO]: reading continuum exposure from lvm-wmflat_ldls-b3.fits
[INFO]: caculating normalization in full wavelength range (3504.35 - 5991.49 angstroms)
[INFO]: computing fiberflat across 1944 fibers and 4080 wavelength bins
[INFO]: applying gaussian smoothing with kernel size 2.5 angstroms (4 pixels)
[WARNING]: at fiber 1: cannot reshape array of size 0 into shape (0,newaxis)
[WARNING]: at fiber 109: cannot reshape array of size 0 into shape (0,newaxis)
[WARNING]: at fiber 289: cannot reshape array of size 0 into shape (0,newaxis)
[WARNING]: at fiber 325: cannot reshape array of size 0 into shape (0,newaxis)
[WARNING]: at fiber 397: cannot reshape array of size 0 into shape (0,newaxis)
[WARNING]: at fiber 414: cannot reshape array of size 0 into shape (0,newaxis)
[WARNING]: at fiber 462: cannot reshape array of size 0 into shape (0,newaxis)
[WARNING]: at fiber

In [4]:
new_masters = md.extract_metadata(frames_paths=mfiberflats)
md.add_masters(new_masters)

[INFO]: going to extract metadata from 9 frames
extracting metadata: 100%|#################################################################################| 9/9 [00:00<00:00, 370.37frame/s]
[INFO]: loading metadata store at /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60147/raw_metadata.hdf5
[INFO]: updating metadata store for tileid = 1111 and mjd = 60147 with 0 new rows
[INFO]: final number of rows 171
[INFO]: writing raw metadata store to disk
[INFO]: loading metadata store at /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/master_metadata.hdf5
[INFO]: updating metadata store for masters with 0 new rows
[INFO]: final number of rows 63
[INFO]: writing master metadata store to disk
